# Sprint17 Faster R-CNNとyolo-V3

# Faster R-CNN

# 問題1

## プログラム準備
https://github.com/duckrabbits/ObjectDetection/tree/master

上記で取得してきたフォルダの中に、このSprint17-rcnn.ipynbを配置します。

## データ準備
+ Kaggleからダウンロードしたthe-simpsons-characters-dataset.zipを解凍します。
+ simpsons_dataset.zipを解凍します。
+ 解凍したsimpsons_datasetフォルダをプロジェクトフォルダ配下に格納します。
+ annotation.txtもプロジェクトフォルダ配下に格納します。 （kaggleからダウンロードしたannotation.txtではなく、ここにあるannotation.txtを使いましょう）

## データ確認

このSprint17-rcnn.ipynbがあるフォルダ内は下記のようになっていると思います。

- kaggle_simpson_testset /（ディレクトリ）
- simpsons_dataset /（ディレクトリ）
- model /（ディレクトリ）
- README.md（ファイル）
- annotation.txt（ファイル）
- predict.py（ファイル）
- train.py（ファイル）
- Sprint17-rcnn.ipynb（ファイル）

## 学習

train.py内の流れとしては、modelフォルダ内の各種プログラムを参照し、モデルを構築し、annotation.txt内の記述に従って、学習用データを取得し学習を実行している

In [8]:
!python train.py -p annotation.txt

Using TensorFlow backend.
Parsing annotation files
Training images per class (19 classes) :
{'abraham_grampa_simpson': 687,
 'apu_nahasapeemapetilon': 206,
 'bart_simpson': 650,
 'bg': 0,
 'charles_montgomery_burns': 650,
 'chief_wiggum': 209,
 'comic_book_guy': 208,
 'edna_krabappel': 212,
 'homer_simpson': 718,
 'kent_brockman': 213,
 'krusty_the_clown': 429,
 'lisa_simpson': 756,
 'marge_simpson': 629,
 'milhouse_van_houten': 210,
 'moe_szyslak': 403,
 'ned_flanders': 675,
 'nelson_muntz': 219,
 'principal_skinner': 614,
 'sideshow_bob': 201}
-------------------------------
path to config file : ./save/train_20210903-152611_config.pickle
-------------------------------









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLaye

## 推測

`-c`のオプションのファイル名は自身の学習実行時に保存されたファイル名を指定してやる必要があります。

In [11]:
!python predict.py -i kaggle_simpson_testset/kaggle_simpson_testset -c ./save/train_20210903-152611_config.pickle

Using TensorFlow backend.










2021-09-03 15:40:19.638808: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-09-03 15:40:19.649782: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe35d0ebc80 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-09-03 15:40:19.649798: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version




abraham_grampa_simpson_0.jpg
[]
Elapsed time = 3.4561727046966553[s]
abraham_grampa_simpson_1.jpg
[]
Elapsed time = 2.5348260402679443[s]
abraham_grampa_simpson_10.jpg
[]
Elapsed time = 2.8572280406951904[s]
abraham_grampa_simpson_11.jpg
[]
Elapsed time = 2.7057299613952637[s]
abraham_grampa_simpson_12.jpg
[]
Elapsed time = 2.5522749423980713[s]
abraham_grampa_simpson_13.jpg
[]
Elapsed time = 2.6604738235473633[s]
abraham_grampa_simpson_14.jpg
[]
Elapsed time 

# 問題2

下記一例です。。。。

RPNは、`resnet.py`の下記で実装されている。

```
def rpn(base_layers,num_anchors):

    x = Convolution2D(512, (3, 3), padding='same', activation='relu', kernel_initializer='normal', name='rpn_conv1')(base_layers)

    x_class = Convolution2D(num_anchors, (1, 1), activation='sigmoid', kernel_initializer='uniform', name='rpn_out_class')(x)
    x_regr = Convolution2D(num_anchors * 4, (1, 1), activation='linear', kernel_initializer='zero', name='rpn_out_regress')(x)

    return [x_class, x_regr, base_layers]
   
```

ROIプーリングは、`RoiPoolingConv.py`の下記で実装されている。

```
class RoiPoolingConv(Layer):
    '''ROI pooling layer for 2D inputs.
    See Spatial Pyramid Pooling in Deep Convolutional Networks for Visual Recognition,
    K. He, X. Zhang, S. Ren, J. Sun
    # Arguments
        pool_size: int
            Size of pooling region to use. pool_size = 7 will result in a 7x7 region.
        num_rois: number of regions of interest to be used
    # Input shape
        list of two 4D tensors [X_img,X_roi] with shape:
        X_img:
        `(1, channels, rows, cols)` if dim_ordering='th'
        or 4D tensor with shape:
        `(1, rows, cols, channels)` if dim_ordering='tf'.
        X_roi:
        `(1,num_rois,4)` list of rois, with ordering (x,y,w,h)
    # Output shape
        3D tensor with shape:
        `(1, num_rois, channels, pool_size, pool_size)`
    '''
    def __init__(self, pool_size, num_rois, **kwargs):

        self.dim_ordering = K.image_data_format()
        assert self.dim_ordering in {'channels_last', 'channels_first'}, 'dim_ordering must be in {channels_last, channels_first}'

        self.pool_size = pool_size
        self.num_rois = num_rois

        super(RoiPoolingConv, self).__init__(**kwargs)

    def build(self, input_shape):
        if self.dim_ordering == 'channels_first':
            self.nb_channels = input_shape[0][1]
        elif self.dim_ordering == 'channels_last':
            self.nb_channels = input_shape[0][3]

    def compute_output_shape(self, input_shape):
        if self.dim_ordering == 'channels_first':
            return None, self.num_rois, self.nb_channels, self.pool_size, self.pool_size
        else:
            return None, self.num_rois, self.pool_size, self.pool_size, self.nb_channels

    def call(self, x, mask=None):

        assert(len(x) == 2)

        img = x[0]
        rois = x[1]

        input_shape = K.shape(img)

        outputs = []

        for roi_idx in range(self.num_rois):

            x = rois[0, roi_idx, 0]
            y = rois[0, roi_idx, 1]
            w = rois[0, roi_idx, 2]
            h = rois[0, roi_idx, 3]

            row_length = w / float(self.pool_size)
            col_length = h / float(self.pool_size)

            num_pool_regions = self.pool_size

            #NOTE: the RoiPooling implementation differs between theano and tensorflow due to the lack of a resize op
            # in theano. The theano implementation is much less efficient and leads to long compile times

            if self.dim_ordering == 'channels_first':
                for jy in range(num_pool_regions):
                    for ix in range(num_pool_regions):
                        x1 = x + ix * row_length
                        x2 = x1 + row_length
                        y1 = y + jy * col_length
                        y2 = y1 + col_length

                        x1 = K.cast(x1, 'int32')
                        x2 = K.cast(x2, 'int32')
                        y1 = K.cast(y1, 'int32')
                        y2 = K.cast(y2, 'int32')

                        x2 = x1 + K.maximum(1,x2-x1)
                        y2 = y1 + K.maximum(1,y2-y1)

                        new_shape = [input_shape[0], input_shape[1],
                                     y2 - y1, x2 - x1]

                        x_crop = img[:, :, y1:y2, x1:x2]
                        xm = K.reshape(x_crop, new_shape)
                        pooled_val = K.max(xm, axis=(2, 3))
                        outputs.append(pooled_val)

            elif self.dim_ordering == 'channels_last':
                x = K.cast(x, 'int32')
                y = K.cast(y, 'int32')
                w = K.cast(w, 'int32')
                h = K.cast(h, 'int32')

                rs = tf.image.resize_images(img[:, y:y+h, x:x+w, :], (self.pool_size, self.pool_size))
                outputs.append(rs)

        final_output = K.concatenate(outputs, axis=0)
        final_output = K.reshape(final_output, (1, self.num_rois, self.pool_size, self.pool_size, self.nb_channels))

        if self.dim_ordering == 'channels_first':
            final_output = K.permute_dimensions(final_output, (0, 1, 4, 2, 3))
        else:
            final_output = K.permute_dimensions(final_output, (0, 1, 2, 3, 4))

        return final_output

```
